In [10]:
from numpy import *

% matplotlib inline

import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

from Orange.data import Table
import collections
from csv import DictReader
from collections import OrderedDict
import pylab

import numpy as np
from sklearn import svm
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from plotting import plot_fit_residual, plot_coefficients
from datetime import date
import time

reader = DictReader(open( "Top5000.csv" , "rt" , encoding="utf-8" ))

podatki=[]
print("STARTING")

def LinearK(x1,y1,x2,y2):
    return (y2-y1/x2-x1)

mc=0
ec=0
gc=0

#Country - Category - JoinDate
for row in reader:
    try:
        datez=str(row["date"])
        yearz=datez.split("-")[0]
        monthz=datez.split("-")[1]
        dayz=datez.split("-")[2]
        datez=date(int(yearz),int(monthz),int(dayz))
        datez=time.mktime(datez.timetuple())
        
        views=row["views"]
        subscribers=row["subscribers"]
        uploads=row["uploads"]
        
        if datez<1420070400:
            pass
        if datez>1451606400:
            continue
        
        country=str(row["country"])
        category=str(row["category"])
        
        if category in ['Games','Entertainment','Music']:
            pass
        else:
            continue
        
        
        if country == "":
            continue
        if category=="":
            continue
        if uploads=="":
            continue
        if views=="":
            continue
        if subscribers=="":
            continue
            
        #newcountry=""
        #Turning US into an integer
        #for c in country:
        #    newcountry+=str(ord(c))
        #country=int(newcountry)
        
        #Turning category into an integer
        #newcategory=""
        #for c in category:
        #    newcategory+=str(ord(c))
        #category=int(newcategory)
        
        
        #KE.append(LinearK(0,datezE[i],PE[i],1451606400))
        
        p=int(subscribers)/int(uploads)
        
        podatki.append([int(uploads),int(subscribers),int(views),int(row["channelId"]),country,category,int(datez),p,LinearK(0,datez,p,1451606400)])

        if category=='Music':
            mc+=1
        if category=='Entertainment':
            ec+=1
        if category=='Games':
            gc+=1
            
    except Exception as e:
        print(e,end='')
        pass

print("END",end="\n\n")

print(len(podatki))
print(mc)
print(ec)
print(gc)

acc=[]
rec=[]
accd=[]
recd=[]
for i in range(0,20):
    np.random.shuffle(podatki)


    #Prvo diskretiziramo datume------------------------------------------------------------------------------

    sorteddata=sorted([d for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    c1=sorteddata[round(length*0.25)]
    c2=sorteddata[round(length/2)]
    c3=sorteddata[round(length*0.75)]
    
    for tup in podatki:
        if tup[6]>c2:
            if tup[6]>c3:
                tup[6]=4
            else:
                tup[6]=3
        else:
            if tup[6]<c1:
                tup[6]=1
            else:
                tup[6]=2
                

    
    #Diskretizacija datumov--END-------------------------------------------------------------------------------

    #Diskretiziramo še po rangu---------------------------------------------------------------------------------

    sorteddata=sorted([a for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    c1=sorteddata[round(length*0.25)]
    c2=sorteddata[round(length/2)]
    c3=sorteddata[round(length*0.75)]

    for tup in podatki:
        if tup[3]>c2:
            if tup[3]>c3:
                tup[3]=4
            else:
                tup[3]=3
        else:
            if tup[3]<c1:
                tup[3]=1
            else:
                tup[3]=2

    #Diskretizacija po rangu--END----------------------------------------------------------------------------------

    #Diskretiziranje za drzave in kategorije----------------------------------------------------------------------------------

    for tup in podatki:

        if tup[4]=='US':
            tup[4]=1;
        else:
            tup[4]=0;

        if tup[5]=='Entertainment':
            tup[5]=1;
        elif tup[5]=='Music':
            tup[5]=2;
        elif tup[5]=='Games':
            tup[5]=3;

    #Diskretiziranje za drzave-in kategorije-END--------------------------------------------------------------------------------
    
    
    #Diskretizacija za Uploads,Subscribers,Views,Perspektivnost,Coeficient--------------------------------------------------------------------------------
    
    sorteddata=sorted([u for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    s1=sorteddata[:round(length*0.125*1)]
    s2=sorteddata[round(length*0.125*1):round(length*0.125*2)]
    s3=sorteddata[round(length*0.125*2):round(length*0.125*3)]
    s4=sorteddata[round(length*0.125*3):round(length*0.125*4)]
    s5=sorteddata[round(length*0.125*4):round(length*0.125*5)]
    s6=sorteddata[round(length*0.125*5):round(length*0.125*6)]
    s7=sorteddata[round(length*0.125*6):round(length*0.125*7)]
    s8=sorteddata[round(length*0.125*7):]
    
    klasses=[s1,s2,s3,s4,s5,s6,s7,s8]
    
    for tup in podatki:
        for i in range(0,len(klasses)):
            if tup[0] in klasses[i]:
                tup[0]=i+1
                break
      
    
    
    sorteddata=sorted([s for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    s1=sorteddata[:round(length*0.125*1)]
    s2=sorteddata[round(length*0.125*1):round(length*0.125*2)]
    s3=sorteddata[round(length*0.125*2):round(length*0.125*3)]
    s4=sorteddata[round(length*0.125*3):round(length*0.125*4)]
    s5=sorteddata[round(length*0.125*4):round(length*0.125*5)]
    s6=sorteddata[round(length*0.125*5):round(length*0.125*6)]
    s7=sorteddata[round(length*0.125*6):round(length*0.125*7)]
    s8=sorteddata[round(length*0.125*7):]
    
    klasses=[s1,s2,s3,s4,s5,s6,s7,s8]
    
    for tup in podatki:
        for i in range(0,len(klasses)):
            if tup[1] in klasses[i]:
                tup[1]=i+1
                break   
                
    sorteddata=sorted([v for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    s1=sorteddata[:round(length*0.125*1)]
    s2=sorteddata[round(length*0.125*1):round(length*0.125*2)]
    s3=sorteddata[round(length*0.125*2):round(length*0.125*3)]
    s4=sorteddata[round(length*0.125*3):round(length*0.125*4)]
    s5=sorteddata[round(length*0.125*4):round(length*0.125*5)]
    s6=sorteddata[round(length*0.125*5):round(length*0.125*6)]
    s7=sorteddata[round(length*0.125*6):round(length*0.125*7)]
    s8=sorteddata[round(length*0.125*7):]
    
    klasses=[s1,s2,s3,s4,s5,s6,s7,s8]
    
    for tup in podatki:
        for i in range(0,len(klasses)):
            if tup[2] in klasses[i]:
                tup[2]=i+1
                break
                
    sorteddata=sorted([p for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    s1=sorteddata[:round(length*0.125*1)]
    s2=sorteddata[round(length*0.125*1):round(length*0.125*2)]
    s3=sorteddata[round(length*0.125*2):round(length*0.125*3)]
    s4=sorteddata[round(length*0.125*3):round(length*0.125*4)]
    s5=sorteddata[round(length*0.125*4):round(length*0.125*5)]
    s6=sorteddata[round(length*0.125*5):round(length*0.125*6)]
    s7=sorteddata[round(length*0.125*6):round(length*0.125*7)]
    s8=sorteddata[round(length*0.125*7):]
    
    klasses=[s1,s2,s3,s4,s5,s6,s7,s8]
    
    for tup in podatki:
        for i in range(0,len(klasses)):
            if tup[7] in klasses[i]:
                tup[7]=i+1
                break
                
    sorteddata=sorted([p for u,s,v,a,b,c,d,p,k in podatki])
    length=len(sorteddata)
    s1=sorteddata[:round(length*0.125*1)]
    s2=sorteddata[round(length*0.125*1):round(length*0.125*2)]
    s3=sorteddata[round(length*0.125*2):round(length*0.125*3)]
    s4=sorteddata[round(length*0.125*3):round(length*0.125*4)]
    s5=sorteddata[round(length*0.125*4):round(length*0.125*5)]
    s6=sorteddata[round(length*0.125*5):round(length*0.125*6)]
    s7=sorteddata[round(length*0.125*6):round(length*0.125*7)]
    s8=sorteddata[round(length*0.125*7):]
    
    klasses=[s1,s2,s3,s4,s5,s6,s7,s8]
    
    for tup in podatki:
        for i in range(0,len(klasses)):
            if tup[8] in klasses[i]:
                tup[8]=i+1
                break
  
    
    #Diskretizacija za Uploads,Subscribers,Views-END----------------------------------------------------------------------------

    splitpoint=round(len(podatki)*0.75)

    learn=podatki[:splitpoint]
    test=podatki[splitpoint:]

    #Predvidevanje ranga na osnovi kategorije in drzave
    #print(learn[:10])
    
    #SETTINGS-------------------------------------------------------------------------------------------------SETTINGS!
    Luvo=[[p,k] for u,s,v,a,b,c,d,p,k in learn]
    Luv=[c for u,s,v,a,b,c,d,p,k in learn]

    Tuvo=[[p,k] for u,s,v,a,b,c,d,p,k in test]

    Tuv=[c for u,s,v,a,b,c,d,p,k in test]



    
    
    clf = GaussianNB()
    #clf=svm.SVC()
    dummy=DummyClassifier(strategy='most_frequent')


    clf.fit(Luvo,Luv)
    dummy.fit(Luvo,Luv)


    predicted=[]
    dummypredicted=[]
    for entry in Tuvo:
        predicted.append(clf.predict(Tuvo)[0])
        dummypredicted.append(dummy.predict(Tuvo)[0])
    
    
    acc.append(accuracy_score(Tuv,predicted))
    rec.append(recall_score(Tuv,predicted))
    
    accd.append(accuracy_score(Tuv,dummypredicted))
    recd.append(recall_score(Tuv,dummypredicted))
    
    #Dobimo povprečje acc ter rec za Naivnega Bayesa
    #DATUM: Predvidevanje datuma glede na vrsto attributov: Vsakič je Naivni bayes bil praktično izenačen z večinskim klasifikatorjem
    #Če smo vzeli več kot 2 attributa pa so ga nove diskretizirane vrednosti zmedle, posledično je natančnost padla pod večinski klass.
    #Še najboljše rezultate smo dosegli z uporabo ranga ter kategorije: Št. poskusov 20
    #NB:
    #0.369805194805
    #0.369805194805
    #Dummy:
    #0.369805194805
    #0.369805194805
    
    #RANG: Predvidevanje razpona ranga glede na datum ter kategorijo/drzavo - zelo podobno:
    #NB:
    #0.246660482375
    #0.246660482375
    #Dummy:
    #0.253153988868
    #0.253153988868
    
    #RANG: Predvidevanje razpona ranga glede na datum:
    #NB:
    #0.24819109462
    #0.24819109462
    #Dummy:
    #0.251901669759
    #0.251901669759
    
    #RANG: Predvidevanje razpona ranga glede na subs,uploads,views X
    #NB:           Do neke mere se da predvidevati vendar to potrjuje da je stvar razporejena po dinamičnosti pridobivanja ogledov.
    #0.25667903525
    #0.25667903525
    #Dummy:
    #0.254359925788
    #0.254359925788
    
    #Predvidevanje subscribers glede na uploads in views X
    #NB:           Bistveno bolse se izkate kar je tudi smiselno. Veliko videov + veliko ogledov = veliko gledalcev
    #0.252690166976
    #0.252690166976
    #Dummy:
    #0.235018552876
    #0.235018552876
    
    #Predvidevanje uploads glede na subs in views X
    #NB:          Enako kot pri zgornjem primeru.
    #0.24633580705
    #0.24633580705
    #Dummy:
    #0.241372912801
    #0.241372912801
    
    #Stevilo subscriberjev iz datuma X
    #NB:
    #0.247680890538
    #0.247680890538
    #Dummy:
    #0.237523191095
    #0.237523191095
    
    #Stevilo viewsou iz datuma X
    #NB:
    #0.252226345083
    #0.252226345083
    #Dummy:
    #0.239332096475
    #0.239332096475
    
    #RANG na osnovi vsega
    #NB:
    #0.248330241187
    #0.248330241187
    #Dummy:
    #0.23594619666
    #0.23594619666
    
    #Razdelil na manjše razrede views,subs in ploads, predvidevanje subs na osnovi views in uploads - zelo podobni rezultati
    #NB:
    #0.125092764378
    #0.125092764378
    #Dummy:
    #0.113636363636
    #0.113636363636
    
    #RANG na osnovi views,subs in uploads -- s tem da imajo zadnji trije 8 razredov vsak
    #NB:
    #0.253617810761
    #0.253617810761
    #Dummy:
    #0.236966604824
    #0.236966604824
    
    #Views na osnovi subs in uploads, svm
    #SVM:
    #0.122854477612
    #0.122854477612
    #Dummy:
    #0.111287313433
    #0.111287313433
    
    #Kategorijo music,entertainment, gaming iz p
    #NB:
    #0.38056768559
    #0.38056768559
    #Dummy:
    #0.367540029112
    #0.367540029112
    
    #Kategorijo music,entertainment,gaming iz k
    #NB:
    #0.341994177584
    #0.341994177584
    #Dummy:
    #0.367467248908
    #0.367467248908
    
    #Kategorijo music,entertainment,gaming iz k p
    #NB:
    #0.355676855895
    #0.355676855895
    #Dummy:
    #0.375327510917
    #0.375327510917
    
    #Kategorijo music,entertainment, gaming iz k p
    #SVM:
    #0.372852983988
    #0.372852983988
    #Dummy:
    #0.369505094614
    #0.369505094614
    
    #Kategorijo music,entertainment,gaming iz p
    #SVM:
    #0.340029112082
    #0.340029112082
    #Dummy:
    #0.369650655022
    #0.369650655022
    
    #Kategorijo music,entertainment,gaming iz k
    #SVM:
    #0.372852983988
    #0.372852983988
    #Dummy:
    #0.368486171761
    #0.368486171761

print("NB:")
print(np.mean(acc))
print(np.mean(rec))
print("Dummy:")
print(np.mean(accd))
print(np.mean(recd))


STARTING
list index out of rangefloat division by zerolist index out of rangelist index out of rangefloat division by zerofloat division by zerolist index out of rangefloat division by zerofloat division by zerofloat division by zerodivision by zerodivision by zerodivision by zerofloat division by zerofloat division by zerofloat division by zerolist index out of rangefloat division by zerofloat division by zerolist index out of rangedivision by zerodivision by zerofloat division by zerofloat division by zerolist index out of rangefloat division by zerolist index out of rangefloat division by zerodivision by zerodivision by zerofloat division by zerolist index out of rangedivision by zerolist index out of rangedivision by zerodivision by zerofloat division by zerodivision by zerofloat division by zerofloat division by zerodivision by zerolist index out of rangelist index out of rangelist index out of rangedivision by zerofloat division by zerofloat division by zerofloat division by zero

c:\python34\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
c:\python34\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
c:\pytho

NB:
0.355676855895
0.355676855895
Dummy:
0.375327510917
0.375327510917


c:\python34\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
c:\python34\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
